In [ ]:
!pip install numpy matplotlib imageio

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

# パラメータの設定
num_particles = 10        # 粒子数
mass = 1.0                # 粒子質量
h = 1.0                   # 平滑化長さ
dt = 0.01                 # 時間ステップ
num_steps = 1000          # シミュレーションステップ数
gamma = 1.4               # 比熱比
rho0 = 1.0                # 初期密度
k = 1.0                   # 圧力定数

# 初期配置の設定
positions = np.random.rand(num_particles, 2)
velocities = np.zeros((num_particles, 2))
densities = np.zeros(num_particles)
pressures = np.zeros(num_particles)
forces = np.zeros((num_particles, 2))

# カーネル関数の定義
def kernel(r, h):
    q = np.linalg.norm(r) / h
    if q <= 1.0:
        return (1.0 - 1.5 * q**2 + 0.75 * q**3) / (np.pi * h**3)
    elif q <= 2.0:
        return 0.25 * (2.0 - q)**3 / (np.pi * h**3)
    else:
        return 0.0

# 密度計算
def compute_densities():
    global densities
    for i in range(num_particles):
        densities[i] = 0.0
        for j in range(num_particles):
            r = positions[i] - positions[j]
            densities[i] += mass * kernel(r, h)

# 圧力計算
def compute_pressures():
    global pressures
    for i in range(num_particles):
        pressures[i] = k * (densities[i] - rho0)

# 力の計算
def compute_forces():
    global forces
    for i in range(num_particles):
        forces[i] = 0.0
        for j in range(num_particles):
            if i != j:
                r = positions[i] - positions[j]
                r_norm = np.linalg.norm(r)
                if r_norm > 0:
                    force_mag = -mass * (pressures[i] / densities[i]**2 + pressures[j] / densities[j]**2) * kernel(r, h)
                    forces[i] += force_mag * r / r_norm

# シミュレーションの実行
def run_simulation():
    global velocities, positions, densities, pressures, forces
    filenames = []
    for step in range(num_steps):
        compute_densities()
        compute_pressures()
        compute_forces()

        # 粒子位置と速度の更新
        velocities += forces * dt / densities[:, np.newaxis]
        positions += velocities * dt

        # 保存量の計算
        total_mass = num_particles * mass
        total_momentum = np.sum(mass * velocities, axis=0)
        kinetic_energy = 0.5 * np.sum(mass * np.sum(velocities**2, axis=1))
        internal_energy = np.sum(mass * pressures / ((gamma - 1) * densities))
        total_energy = kinetic_energy + internal_energy

        # 保存量の表示
        print(f'Step: {step}, Total Mass: {total_mass}, Total Momentum: {total_momentum}, Total Energy: {total_energy}')

        # 可視化と画像の保存
        if step % 1 == 0:  # すべてのステップで画像を保存
            plt.scatter(positions[:, 0], positions[:, 1], s=10)
            plt.xlim(0, 1)
            plt.ylim(0, 1)
            plt.title(f'Step: {step}')
            filename = f'step_{step}.png'
            filenames.append(filename)
            plt.savefig(filename)
            plt.close()

    # GIFの作成
    with imageio.get_writer('simulation.gif', mode='I', duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # 一時ファイルの削除
    for filename in filenames:
        os.remove(filename)

# シミュレーションの実行
run_simulation()


Step: 0, Total Mass: 10.0, Total Momentum: [-0.00050264  0.00026415], Total Energy: 14.38601916958586
Step: 1, Total Mass: 10.0, Total Momentum: [-0.00100522  0.0005283 ], Total Energy: 14.386598767418802
Step: 2, Total Mass: 10.0, Total Momentum: [-0.00150769  0.00079244], Total Energy: 14.387718636724596
Step: 3, Total Mass: 10.0, Total Momentum: [-0.00200997  0.00105657], Total Energy: 14.389378497833652
Step: 4, Total Mass: 10.0, Total Momentum: [-0.00251201  0.00132068], Total Energy: 14.391577931009204
Step: 5, Total Mass: 10.0, Total Momentum: [-0.00301376  0.00158478], Total Energy: 14.394316376515368
Step: 6, Total Mass: 10.0, Total Momentum: [-0.00351515  0.00184886], Total Energy: 14.397593134709151
Step: 7, Total Mass: 10.0, Total Momentum: [-0.00401612  0.00211291], Total Energy: 14.40140736615675
Step: 8, Total Mass: 10.0, Total Momentum: [-0.00451661  0.00237694], Total Energy: 14.40575809177439
Step: 9, Total Mass: 10.0, Total Momentum: [-0.00501657  0.00264093], Total 

<ipython-input-12-775fa5d23818>:98: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

# パラメータの設定
num_particles = 100      # 粒子数
mass = 1.0               # 粒子質量
h = 1.0                  # 平滑化長さ
dt = 0.01                # 時間ステップ
num_steps = 100          # シミュレーションステップ数
gamma = 1.4              # 比熱比
rho0 = 1.0               # 初期密度
k = 1.0                  # 圧力定数
G = 1.0                  # 重力定数

# 初期配置の設定
positions = np.random.rand(num_particles, 2)
velocities = np.zeros((num_particles, 2))
densities = np.zeros(num_particles)
pressures = np.zeros(num_particles)
forces = np.zeros((num_particles, 2))

# カーネル関数の定義
def kernel(r, h):
    q = np.linalg.norm(r) / h
    if q <= 1.0:
        return (1.0 - 1.5 * q**2 + 0.75 * q**3) / (np.pi * h**3)
    elif q <= 2.0:
        return 0.25 * (2.0 - q)**3 / (np.pi * h**3)
    else:
        return 0.0

# 密度計算
def compute_densities():
    global densities
    for i in range(num_particles):
        densities[i] = 0.0
        for j in range(num_particles):
            r = positions[i] - positions[j]
            densities[i] += mass * kernel(r, h)

# 圧力計算
def compute_pressures():
    global pressures
    for i in range(num_particles):
        pressures[i] = k * (densities[i] - rho0)

# 力の計算
def compute_forces():
    global forces
    for i in range(num_particles):
        forces[i] = 0.0
        for j in range(num_particles):
            if i != j:
                r = positions[i] - positions[j]
                r_norm = np.linalg.norm(r)
                if r_norm > 0:
                    force_mag = -mass * (pressures[i] / densities[i]**2 + pressures[j] / densities[j]**2) * kernel(r, h)
                    forces[i] += force_mag * r / r_norm

# 自己重力の計算
def compute_gravity():
    global forces
    for i in range(num_particles):
        for j in range(num_particles):
            if i != j:
                r = positions[j] - positions[i]
                r_norm = np.linalg.norm(r)
                if r_norm > 0:
                    force_mag = G * mass**2 / r_norm**3
                    forces[i] += force_mag * r

# シミュレーションの実行
def run_simulation():
    global velocities, positions, densities, pressures, forces
    filenames = []
    for step in range(num_steps):
        compute_densities()
        compute_pressures()
        compute_forces()
        compute_gravity()

        # 粒子位置と速度の更新
        velocities += forces * dt / densities[:, np.newaxis]
        positions += velocities * dt

        # 保存量の計算
        total_mass = num_particles * mass
        total_momentum = np.sum(mass * velocities, axis=0)
        kinetic_energy = 0.5 * np.sum(mass * np.sum(velocities**2, axis=1))
        internal_energy = np.sum(mass * pressures / ((gamma - 1) * densities))
        total_energy = kinetic_energy + internal_energy

        # 保存量の表示
        print(f'Step: {step}, Total Mass: {total_mass}, Total Momentum: {total_momentum}, Total Energy: {total_energy}')

        # 可視化と画像の保存
        if step % 1 == 0:  # すべてのステップで画像を保存
            plt.scatter(positions[:, 0], positions[:, 1], s=10)
            plt.xlim(0, 1)
            plt.ylim(0, 1)
            plt.title(f'Step: {step}')
            filename = f'step_{step}.png'
            filenames.append(filename)
            plt.savefig(filename)
            plt.close()

    # GIFの作成
    with imageio.get_writer('simulation.gif', mode='I', duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # 一時ファイルの削除
    for filename in filenames:
        os.remove(filename)

# シミュレーションの実行
run_simulation()


Step: 0, Total Mass: 100.0, Total Momentum: [0.09535337 0.04695388], Total Energy: 325.3044069063791
Step: 1, Total Mass: 100.0, Total Momentum: [ 0.07906537 -0.04161259], Total Energy: 64648.51841474758
Step: 2, Total Mass: 100.0, Total Momentum: [0.30051414 0.10851344], Total Energy: 64895.85026128557
Step: 3, Total Mass: 100.0, Total Momentum: [0.28001471 0.14475658], Total Energy: 64860.84769088581
Step: 4, Total Mass: 100.0, Total Momentum: [0.08026354 0.11180569], Total Energy: 65183.92892550026
Step: 5, Total Mass: 100.0, Total Momentum: [-0.28179185  0.18441578], Total Energy: 68148.21224340802
Step: 6, Total Mass: 100.0, Total Momentum: [ 0.34918586 -0.40163671], Total Energy: 4737290.667123412
Step: 7, Total Mass: 100.0, Total Momentum: [ 0.22886264 -0.35442026], Total Energy: 4737275.582783523
Step: 8, Total Mass: 100.0, Total Momentum: [-0.06427536 -0.16338988], Total Energy: 4739176.713099468
Step: 9, Total Mass: 100.0, Total Momentum: [-0.15500281 -0.064578  ], Total Ener

<ipython-input-13-07674f2d590e>:112: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

# パラメータの設定
num_particles = 1000     # 粒子数
mass = 1.0               # 粒子質量
h = 1.0                  # 平滑化長さ
dt = 0.01                # 時間ステップ
num_steps = 200          # シミュレーションステップ数
gamma = 1.4              # 比熱比
rho0 = 1.0               # 初期密度
k = 1.0                  # 圧力定数

# 初期配置の設定
positions = np.random.rand(num_particles, 2) * 10.0  # 広い領域に分布させるため、2倍の範囲に設定
# 初期速度の設定
velocities = np.zeros((num_particles, 2))
colors = []
for i in range(num_particles):
    if positions[i, 1] > 1.0:
        velocities[i, 0] = 0.5  # 上部の流れ
        colors.append('blue')
    else:
        velocities[i, 0] = -0.5  # 下部の流れ
        colors.append('red')
    # KH不安定性を引き起こすための小さな摂動を追加
    velocities[i, 1] += 0.01 * (np.random.rand() - 0.5)

densities = np.zeros(num_particles)
pressures = np.zeros(num_particles)
forces = np.zeros((num_particles, 2))

# カーネル関数の定義
def kernel(r, h):
    q = np.linalg.norm(r) / h
    if q <= 1.0:
        return (1.0 - 1.5 * q**2 + 0.75 * q**3) / (np.pi * h**3)
    elif q <= 2.0:
        return 0.25 * (2.0 - q)**3 / (np.pi * h**3)
    else:
        return 0.0

# 密度計算
def compute_densities():
    global densities
    for i in range(num_particles):
        densities[i] = 0.0
        for j in range(num_particles):
            r = positions[i] - positions[j]
            densities[i] += mass * kernel(r, h)

# 圧力計算
def compute_pressures():
    global pressures
    for i in range(num_particles):
        pressures[i] = k * (densities[i] - rho0)

# 力の計算
def compute_forces():
    global forces
    for i in range(num_particles):
        forces[i] = 0.0
        for j in range(num_particles):
            if i != j:
                r = positions[i] - positions[j]
                r_norm = np.linalg.norm(r)
                if r_norm > 0:
                    force_mag = -mass * (pressures[i] / densities[i]**2 + pressures[j] / densities[j]**2) * kernel(r, h)
                    forces[i] += force_mag * r / r_norm

# シミュレーションの実行
def run_simulation():
    global velocities, positions, densities, pressures, forces, colors
    filenames = []
    for step in range(num_steps):
        compute_densities()
        compute_pressures()
        compute_forces()

        # 粒子位置と速度の更新
        velocities += forces * dt / densities[:, np.newaxis]
        positions += velocities * dt

        # 保存量の計算
        total_mass = num_particles * mass
        total_momentum = np.sum(mass * velocities, axis=0)
        kinetic_energy = 0.5 * np.sum(mass * np.sum(velocities**2, axis=1))
        internal_energy = np.sum(mass * pressures / ((gamma - 1) * densities))
        total_energy = kinetic_energy + internal_energy

        # 保存量の表示
        print(f'Step: {step}, Total Mass: {total_mass}, Total Momentum: {total_momentum}, Total Energy: {total_energy}')

        # 可視化と画像の保存
        if step % 1 == 0:  # すべてのステップで画像を保存
            plt.scatter(positions[:, 0], positions[:, 1], c=colors, s=10)
            plt.xlim(0, 2)
            plt.ylim(0, 2)
            plt.title(f'Step: {step}')
            filename = f'step_{step}.png'
            filenames.append(filename)
            plt.savefig(filename)
            plt.close()

    # GIFの作成
    with imageio.get_writer('KHsimulation.gif', mode='I', duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # 一時ファイルの削除
    for filename in filenames:
        os.remove(filename)

# シミュレーションの実行
run_simulation()


Step: 0, Total Mass: 1000.0, Total Momentum: [4.02004970e+02 6.35704822e-02], Total Energy: 2232.522059596024
Step: 1, Total Mass: 1000.0, Total Momentum: [4.02009921e+02 6.03023111e-02], Total Energy: 2232.5426826212765
Step: 2, Total Mass: 1000.0, Total Momentum: [4.02014853e+02 5.70173402e-02], Total Energy: 2232.5654981941216
Step: 3, Total Mass: 1000.0, Total Momentum: [4.02019766e+02 5.37161030e-02], Total Energy: 2232.5905052105486
Step: 4, Total Mass: 1000.0, Total Momentum: [4.02024658e+02 5.03991309e-02], Total Energy: 2232.617702543038
Step: 5, Total Mass: 1000.0, Total Momentum: [4.02029529e+02 4.70669540e-02], Total Energy: 2232.6470887436394
Step: 6, Total Mass: 1000.0, Total Momentum: [4.02034380e+02 4.37201014e-02], Total Energy: 2232.67866231985
Step: 7, Total Mass: 1000.0, Total Momentum: [4.02039208e+02 4.03591030e-02], Total Energy: 2232.7124218525573
Step: 8, Total Mass: 1000.0, Total Momentum: [4.02044015e+02 3.69844899e-02], Total Energy: 2232.7483665275845
Step: